In [1]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [2]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsusb"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="05e6", ATTRS{idProduct}=="6500" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

rm = pyvisa.ResourceManager()
print(rm.list_resources('?*'))

inst = rm.open_resource('ASRL3::INSTR')
inst.read_termination = '\r'
inst.write_termination = '\r'
inst.baud_rate = 19200

inst.write('*IDN?')
print(inst.read())

('ASRL3::INSTR',)
KEITHLEY INSTRUMENTS INC.,MODEL 6517B,4097463,A13/700x 


In [81]:
inst.write('SENS:FUNC "VOLT:DC"')
inst.write('SENS:VOLT:DC:RANG 0.000001')
inst.write('SENS:VOLT:DC:NPLC 1')
#inst.write('SENS:VOLT:REFerence:state OFF')

30

In [69]:
inst.write('SENS:FUNC "CURR:DC"')
inst.write('SENS:CURR:DC:RANG 0.00000000001')
inst.write('SENS:CURR:DC:NPLC 1')


20

In [61]:
inst.write('*CLS')
inst.write('*RST')
inst.write('*IDN?')
print(inst.read())
inst.write('SYSTem:ZCHeck OFF')
inst.write('CALCulate:State OFF')

KEITHLEY INSTRUMENTS INC.,MODEL 6517B,4097463,A13/700x 


20

In [62]:
inst.write('INITiate:CONTinuous ON')

23

In [63]:
def query(s):
    inst.write(s)
    # collect return
    out =''
    while True:
        line = inst.read_bytes(1)
        if line == b'1`':
            if len(out)>0:
                break
            continue
        out += line.decode('utf-8')
        if b'\r' in line: # newline marks end of return
            break
    return out

In [64]:
query('SENS:DATA:FRES?')

'+00.11716E+00NVDC,04:45:09.00,13-Sep-2022,+00311RDNG#\r'

In [106]:
rate = 10
samples = 0
def aquire_as_thread():
    global samples
    while keep_alive:
        s = str(datetime.now()) + '\n'
        for i in range(rate):
            s = s+query('SENS:DATA:FRES?').split(',')[0]
            time.sleep(1/rate)
            if i<rate-1:
                s = s + ' '
        with open('electrometer_log.txt','a') as fi:
            fi.write(s+'\n')
        samples += 1
        if keep_alive:
            print(f'electrometer {samples}x{rate} datapoints', end = '\r')
        else:
            break
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [107]:
# to stop reading thread:
if 1:
    keep_alive = False
    print(f'multimeter {samples}x{rate} datapoints')

multimeter 2x10 datapoints


In [3]:
import os
def get_last_lines(file, n):
    with open(file, 'rb') as f:
        if os.path.getsize(file) > 1000:
            f.seek(-1000, 2)
        data = f.readlines()
        lines = []
        for i in range(n,0,-1):
            lines.append(data[-i].decode("utf-8"))
        lines = ' / '.join(lines).replace('\r\n', '').strip()
    return lines

In [4]:
lines = get_last_lines('electrometer_log.txt', 2)
print(lines)

FileNotFoundError: [Errno 2] No such file or directory: 'electrometer_log.txt'